In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StandardScaler
from pyspark.sql.types import IntegerType, FloatType
from pyspark.sql.functions import col
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
sprk_session=spark_session = SparkSession.builder.master('local[*]').appName('Admission_pre').getOrCreate()

    

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/23 17:11:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df=spark_session.read.csv("Admission_Predict.csv",header=True)

In [4]:
df.cache()
df.printSchema()

root
 |-- Serial No.: string (nullable = true)
 |-- GRE Score: string (nullable = true)
 |-- TOEFL Score: string (nullable = true)
 |-- University Rating: string (nullable = true)
 |-- SOP: string (nullable = true)
 |-- LOR : string (nullable = true)
 |-- CGPA: string (nullable = true)
 |-- Research: string (nullable = true)
 |-- Chance of Admit : string (nullable = true)



In [5]:
df.show(3)

+----------+---------+-----------+-----------------+---+----+----+--------+----------------+
|Serial No.|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+----------+---------+-----------+-----------------+---+----+----+--------+----------------+
|         1|      337|        118|                4|4.5| 4.5|9.65|       1|            0.92|
|         2|      324|        107|                4|  4| 4.5|8.87|       1|            0.76|
|         3|      316|        104|                3|  3| 3.5|   8|       1|            0.72|
+----------+---------+-----------+-----------------+---+----+----+--------+----------------+
only showing top 3 rows



In [6]:
cols=df.columns
cols.remove('Serial No.')
cols

['GRE Score',
 'TOEFL Score',
 'University Rating',
 'SOP',
 'LOR ',
 'CGPA',
 'Research',
 'Chance of Admit ']

In [7]:
for column in cols:
    df=df.withColumn(column,col(column).cast(FloatType()))
df.printSchema()

root
 |-- Serial No.: string (nullable = true)
 |-- GRE Score: float (nullable = true)
 |-- TOEFL Score: float (nullable = true)
 |-- University Rating: float (nullable = true)
 |-- SOP: float (nullable = true)
 |-- LOR : float (nullable = true)
 |-- CGPA: float (nullable = true)
 |-- Research: float (nullable = true)
 |-- Chance of Admit : float (nullable = true)



In [8]:
vector_assembler=VectorAssembler(inputCols=['GRE Score', 'TOEFL Score', 'University Rating', 'SOP', 'LOR ' , 'CGPA', 'Research'],outputCol='features')
df_vect=vector_assembler.transform(df)
df_vect.select(['features', 'Chance of Admit ']).show(5)
df_train=df_vect.select(['features', 'Chance of Admit '])

+--------------------+----------------+
|            features|Chance of Admit |
+--------------------+----------------+
|[337.0,118.0,4.0,...|            0.92|
|[324.0,107.0,4.0,...|            0.76|
|[316.0,104.0,3.0,...|            0.72|
|[322.0,110.0,3.0,...|             0.8|
|[314.0,103.0,2.0,...|            0.65|
+--------------------+----------------+
only showing top 5 rows



In [9]:
train_df, test_df= df_train.randomSplit([0.75, 0.25])
train_df.show(4)

+--------------------+----------------+
|            features|Chance of Admit |
+--------------------+----------------+
|[290.0,104.0,4.0,...|            0.45|
|[293.0,97.0,2.0,2...|            0.64|
|[294.0,93.0,1.0,1...|            0.46|
|[294.0,95.0,1.0,1...|            0.49|
+--------------------+----------------+
only showing top 4 rows



In [10]:
standard_scaler= StandardScaler(inputCol='features', outputCol='Scaled_features')
standard_scaler_model = standard_scaler.fit(train_df)
train_df_transform = standard_scaler_model.transform(train_df)
test_df_transform = standard_scaler_model.transform(test_df)
train_df_transform.take(4)

[Row(features=DenseVector([290.0, 104.0, 4.0, 2.0, 2.5, 7.46, 0.0]), Chance of Admit =0.44999998807907104, Scaled_features=DenseVector([25.1466, 16.8299, 3.4858, 2.0302, 2.8317, 12.4339, 0.0])),
 Row(features=DenseVector([293.0, 97.0, 2.0, 2.0, 4.0, 7.8, 1.0]), Chance of Admit =0.6399999856948853, Scaled_features=DenseVector([25.4068, 15.6972, 1.7429, 2.0302, 4.5307, 13.0006, 2.0105])),
 Row(features=DenseVector([294.0, 93.0, 1.0, 1.5, 2.0, 7.36, 0.0]), Chance of Admit =0.46000000834465027, Scaled_features=DenseVector([25.4935, 15.0499, 0.8715, 1.5227, 2.2653, 12.2672, 0.0])),
 Row(features=DenseVector([294.0, 95.0, 1.0, 1.5, 1.5, 7.64, 0.0]), Chance of Admit =0.49000000953674316, Scaled_features=DenseVector([25.4935, 15.3735, 0.8715, 1.5227, 1.699, 12.7339, 0.0]))]

In [11]:
lr = LinearRegression(featuresCol = 'Scaled_features', labelCol='Chance of Admit ')
lr_model=lr.fit(train_df_transform)

23/03/23 17:12:07 WARN Instrumentation: [07ac480b] regParam is zero, which might cause numerical instability and overfitting.
23/03/23 17:12:08 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/03/23 17:12:08 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
23/03/23 17:12:08 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [12]:
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [0.023111034659754778,0.018159370059506602,0.0021027545642635444,0.0013207337526784284,0.016034360120147324,0.06545759465353468,0.012604138306556887]
Intercept: -1.2473544158423178


In [13]:
lr_prediction_test=lr_model.transform(test_df_transform)
lr_prediction_train=lr_model.transform(train_df_transform)
lr_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="Chance of Admit ",metricName="r2")

print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_prediction_test))
print("R Squared (R2) on train data = %g" % lr_evaluator.evaluate(lr_prediction_train))

R Squared (R2) on test data = 0.802028
R Squared (R2) on train data = 0.79705
